### Regresión Logística - Pair programming ###

## 2. Preprocesado ##

In [ ]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd


# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


# Estandarización variables numéricas y Codificación variables categóricas
# -----------------------------------------------------------------------------
from sklearn.preprocessing import RobustScaler

# Gestión datos desbalanceados
# ------------------------------------------------------------------------------
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek

# Para separar los datos en train y test
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split

#  Gestión de warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

Usando el mismo dataset que usatéis ayer, los objetivos de los ejercicios de hoy son:

Estandarizar las variables numéricas de vuestro set de datos

Codificar las variables categóricas. Recordad que tendréis que tener en cuenta si vuestras variables tienen orden o no.

Chequear si vuestros datos están balanceados. En caso de que no lo estén utilizad algunas de las herramientas aprendidas en la lección para balancearlos.

Guardad el dataframe con los cambios que habéis aplicado para utilizarlo en la siguiente lección.

In [ ]:
df = pd.read_pickle("data/card_limpio.pkl")
df.head()

In [ ]:
df.info()

Vamos a proceder a estandarizar los datos, como tenemos outliers lo haremos con el método RobustScaler.

In [ ]:
numericas = df.select_dtypes(include = np.number)

In [ ]:
numericas.head()

In [ ]:
# construir el modelo de escalador
robust = RobustScaler()

In [ ]:
# ajustamos el modelo utilizando nuestro set de datos
robust.fit(numericas)

In [ ]:
# transformamos los datos
X_robust = robust.transform(numericas)

In [ ]:
numericas_robust = pd.DataFrame(X_robust, columns = numericas.columns)
numericas_robust.head(2)

In [ ]:
for columna in numericas_robust.columns:
    print(f'La media de la columna {columna} es:  {numericas_robust[columna].mean()}')
    print(f'La desviación estándar de la columna {columna} es: {numericas_robust[columna].std()}')
    print("-------------------------------------------------------------")

In [ ]:
fig, axes = plt.subplots(nrows = 3, ncols = 1, figsize = (30, 10))

columnas_robust = numericas_robust.columns
axes = axes.flat


for i, colum in enumerate(columnas_robust): 
    sns.histplot(
        data = numericas_robust,
        x = colum,
        kde = True,  
        line_kws = {"linewidth": 2}, 
        alpha = 0.2, 
        ax = axes[i])
    
    
    axes[i].set_title(colum, fontsize = 15, fontweight = "bold")
    axes[i].tick_params(labelsize = 20)
    axes[i].set_xlabel("")

fig.tight_layout()


In [ ]:
df[numericas_robust.columns] = numericas_robust
df.head()

Procedemos a hacer el encoding de las variables categóricas. Vamos a ver el orden:

    - repeat_retailer	**SI** tiene orden
    - used_chip	    **SI** tiene orden
    - used_pin_number	**NO** tiene orden
    - online_order  **SI** tiene orden

Entonces las columnas que tienen orden las dejamos como están puesto que son binarias, y la columna que no tiene orden (used_pin_number) le vamos a aplicar get_dummies.

In [ ]:
# dummies = pd.get_dummies(df["used_pin_number"], prefix_sep = "_", prefix = "used_pin_number", dtype = int)
# dummies.head(2)

In [ ]:
# df_est_enc = pd.concat([df, dummies], axis = 1)
# df_est_enc.drop('used_pin_number', axis = 1, inplace= True)

In [ ]:
# df_est_enc.head()

In [ ]:
df.to_pickle('data/card_est_enc.pkl')

In [ ]:
# para aplicar este método lo primero que tenemos que hacer es separar en X e y y en train y test como aprendimos en la lecciones de regresion lineal
y = df['fraud']
X = df.drop('fraud', axis=1)


#dividimos en sets de entrenamiento y test
#X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [ ]:
# iniciamos el método
os_us = SMOTETomek()

# ajustamos el modelo
X_train_res, y_train_res = os_us.fit_resample(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [ ]:
y_train.value_counts()

In [ ]:
y_train_res.value_counts()

In [ ]:
# comprobemos como han quedado ahora las categorías después del ajuste

print (f"Distribution before resampling \n {y_train.value_counts()}" )
print("..............................................................")
print (f"Distribución después del ajuste \n {y_train_res.value_counts()}")

In [ ]:
df.info()

In [ ]:
y_train_res.shape

In [ ]:
y_train_res.shape

In [ ]:
X_train_res.isnull().sum()

In [ ]:
df_balanceo = pd.concat([y_train_res, X_train_res], axis = 1)

In [ ]:
df_balanceo.isnull().sum()

In [ ]:
 df_balanceo.dropna(inplace = True)

In [ ]:
df_balanceo['fraud'].value_counts()

In [ ]:
df_balanceo.to_pickle('data/df_balanceo.pkl')